In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
import json
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)
from multilayer_model.m2_bce import BCEcVAE
from multilayer_model.m2_mse import MSEcVAE
from multilayer_loss.l2_bce import l2_bce
from multilayer_loss.l2_mse import l2_mse
from vae_earlystopping import EarlyStopping

In [3]:


# 결과 저장용 리스트
r2_results = []
num_runs = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 데이터 로드 (루프 밖에서 한 번만 수행)
x_data = np.load('./data/metal.npy')
x2_data = np.load('./data/support_norm.npy')
c_data = np.load('./data/pre_re_change_temp_logconst.npy')

print(f"Data loaded. Starting {num_runs} independent runs...")

for run in range(1, num_runs + 1):
    print(f"\n--- Run {run}/{num_runs} ---")
    
    # 2. 데이터 분할 (매번 동일한 seed를 피하고 싶다면 random_state를 run으로 설정 가능)
    c_train, c_temp, x_train, x_temp, x2_train, x2_temp = train_test_split(
        c_data, x_data, x2_data, test_size=0.4, random_state=run
    )
    c_val, c_test, x_val, x_test, x2_val, x2_test = train_test_split(
        c_temp, x_temp, x2_temp, test_size=0.5, random_state=run
    )

    # 3. 스케일링
    c_scaler, x_scaler, x2_scaler = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
    c_train = c_scaler.fit_transform(c_train)
    x_train = x_scaler.fit_transform(x_train)
    x2_train = x2_scaler.fit_transform(x2_train)
    
    c_val, x_val, x2_val = c_scaler.transform(c_val), x_scaler.transform(x_val), x2_scaler.transform(x2_val)
    c_test, x_test, x2_test = c_scaler.transform(c_test), x_scaler.transform(x_test), x2_scaler.transform(x2_test)

    # 4. 텐서 변환 및 데이터로더 생성
    def to_tensor(arr): return torch.tensor(arr, dtype=torch.float32)
    
    train_loader = DataLoader(TensorDataset(to_tensor(x_train), to_tensor(x2_train), to_tensor(c_train)), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(to_tensor(x_val), to_tensor(x2_val), to_tensor(c_val)), batch_size=64, shuffle=False)
    test_loader = DataLoader(TensorDataset(to_tensor(x_test), to_tensor(x2_test), to_tensor(c_test)), batch_size=64, shuffle=False)

    # 차원 설정
    x_dim, x2_dim, c_dim = x_train.shape[1], x2_train.shape[1], c_train.shape[1]
    z_dim, z2_dim = 8, 8

    # 5. 모델 및 옵티마이저 초기화
    model_bce = BCEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
    model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
    
    optimizer_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
    optimizer_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
    
    es_bce = EarlyStopping(patience=40, min_delta=1e-9)
    es_mse = EarlyStopping(patience=40, min_delta=1e-9)

    # --- Phase 1: BCE 모델 학습 ---
    for epoch in range(1, 801):
        model_bce.train()
        for x, x2, c in train_loader:
            x, x2, c = x.to(device), x2.to(device), c.to(device)
            optimizer_bce.zero_grad()
            logit, mu, logvar = model_bce(x, x2, c)
            loss = l2_bce(logit, x2, mu, logvar)['loss']
            loss.backward()
            optimizer_bce.step()
        
        model_bce.eval()
        v_loss = 0
        with torch.no_grad():
            for vx, vx2, vc in val_loader:
                vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
                v_logit, v_mu, v_logvar = model_bce(vx, vx2, vc)
                v_loss += l2_bce(v_logit, vx2, v_mu, v_logvar)['loss'].item()
        
        avg_v_loss = v_loss / len(val_loader)
        if es_bce(avg_v_loss, model_bce): break
    
    es_bce.load_best_model(model_bce)

    # --- Phase 2: MSE 모델 학습 ---
    for epoch in range(1, 801):
        model_mse.train()
        for x, x2, c in train_loader:
            x, x2, c = x.to(device), x2.to(device), c.to(device)
            optimizer_mse.zero_grad()
            x_hat, mu, logvar = model_mse(x, x2, c)
            loss = l2_mse(x_hat, x2, mu, logvar)['loss']
            loss.backward()
            optimizer_mse.step()

        model_mse.eval()
        v_loss = 0
        with torch.no_grad():
            for vx, vx2, vc in val_loader:
                vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
                vh, vm, vl = model_mse(vx, vx2, vc)
                v_loss += l2_mse(vh, vx2, vm, vl)['loss'].item()
        
        avg_v_loss = v_loss / len(val_loader)
        if es_mse(avg_v_loss, model_mse): break

    es_mse.load_best_model(model_mse)

    # --- 최종 평가 (Test Set) ---
    model_bce.eval()
    model_mse.eval()
    all_pred, all_true = [], []

    with torch.no_grad():
        for xt, xt2, ct in test_loader:
            xt, xt2, ct = xt.to(device), xt2.to(device), ct.to(device)
            b_logit, _, _ = model_bce(xt, xt2, ct)
            b_prob = torch.sigmoid(b_logit)
            m_val, _, _ = model_mse(xt, xt2, ct)
            
            combined_s = b_prob * m_val
            all_pred.append(combined_s.cpu().numpy())
            all_true.append(xt2.cpu().numpy()) # 타겟은 x2

    y_true = np.concatenate(all_true).ravel()
    y_pred = np.concatenate(all_pred).ravel()
    r2 = r2_score(y_true, y_pred)
    r2_results.append(r2)
    print(f"Run {run} R2: {r2:.6f}")

# --- 6. 통계 저장 ---
results_json = {
    "mean_r2": float(np.mean(r2_results)),
    "std_r2": float(np.std(r2_results)),
    "all_r2": [float(x) for x in r2_results]
}

with open("r2_statistics.json", "w") as f:
    json.dump(results_json, f, indent=4)

print("\nFinal Statistics Saved.")
print(f"Mean R2: {results_json['mean_r2']:.6f} ± {results_json['std_r2']:.6f}")

Data loaded. Starting 20 independent runs...

--- Run 1/20 ---
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 ou